# Projeto de TCC
**Autor**: Igor Sousa dos Santos Santana

**Data**: 2024-07-26

## Índice

- [Importando pacotes e Bibliotecas](#importando-pacotes-e-bibliotecas)
- [Importando os Datasets](#importando-os-datasets)
- [Breve Apresentação dos Datasets](#breve-apresentacao-dos-datasets)
- [Breve Limpeza nos Datasets](./limpeza_dados.ipynb)
- [Análise Exploratória de Dados](./analise_exploratoria.ipynb)
- [Preparação dos parâmetros](#preparando-os-hiper-parâmetros)
- [Algoritmo KMeans](#utilizando-o-algoritmo-kmeans)
- [Algoritmo MeanShift](#utilizando-o-algoritmo-meanshift)
- [Algoritmo FuzzyCMeans](#usando-o-algoritmo-fuzzycmeans)
- [Bibliografia](#bibliografia)

## Importando pacotes e Bibliotecas

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from models import kmeans, meanshift
from fcmeans import FCM
import seaborn as sns
import pandas as pd
import numpy as np

## Importando os Datasets

In [ ]:
df_iris_raw = pd.read_csv("./databases/raw/Iris.csv", sep = ",", index_col = "Id")
df_titanic_processed = pd.read_pickle("./databases/processed/titanic_processado.pkl")
df_coracao_processed = pd.read_pickle("./databases/processed/coracao_processado.pkl")

## Breve apresentacao dos datasets

In [ ]:
df_iris_raw.sample(10)

In [ ]:
df_titanic_processed.sample(10)

In [ ]:
df_coracao_processed.sample(10)

In [ ]:
df_iris_raw.info()

In [ ]:
df_titanic_processed.info()

In [ ]:
df_coracao_processed.info()

## Preparando os hiper parâmetros

In [ ]:
X_iris = df_iris_raw.drop(columns = ["Species"]).values
y_iris = df_iris_raw["Species"]

min_max_scaler = MinMaxScaler()
X_titanic = min_max_scaler.fit_transform(df_titanic_processed.drop(columns = ["Pclass", "Name"]).values)
y_titanic = df_titanic_processed["Pclass"]

In [ ]:
nro_max_iteracoes = 300
nro_iteracoes_inicial = 10
semente_randomica = 42
parametro_fuzzy = 2

## Utilizando o algoritmo Kmeans

In [ ]:
soma_dos_quadrados_iris = []
soma_dos_quadrados_titanic = []

for i in range (1, 14):
    kmeans = KMeans(n_clusters = i,
                    init = "k-means++",
                    max_iter = nro_max_iteracoes,
                    n_init = nro_iteracoes_inicial,
                    random_state = semente_randomica)
    kmeans.fit(X_iris)
    soma_dos_quadrados_iris.append(kmeans.inertia_)
    
    kmeans = KMeans(n_clusters = i,
                    init = "k-means++",
                    max_iter = nro_max_iteracoes,
                    n_init = nro_iteracoes_inicial,
                    random_state = semente_randomica)
    kmeans.fit(X_titanic)
    soma_dos_quadrados_titanic.append(kmeans.inertia_)

### Imprimindo o resultado dos testes

In [ ]:
figura, eixos = plt.subplots(1, 2, figsize = (18, 5))

plt.suptitle("Método do joelho")
plt.xlabel("Número de clusters")
plt.ylabel("Soma dos Quadrados Internos ao Clusters")

eixos[0].plot(range(1, 14), soma_dos_quadrados_iris, color = "red")
eixos[1].plot(range(1, 14), soma_dos_quadrados_titanic, color = "blue")

plt.show()

No caso acima, o resultado a partir do número 3 decai muito, portanto o número de clusters escolhido é:
- `3` para o dataset `Iris`;
- `3` para o dataset `Titanic`.

### Implementando a clusterização

#### Dataset Iris

In [ ]:
kmeans_iris = KMeans(n_clusters = 3,
                    init = "k-means++",
                    max_iter = nro_max_iteracoes,
                    n_init = nro_iteracoes_inicial,
                    random_state = semente_randomica)
y_kmeans_iris = kmeans_iris.fit_predict(X_iris)

#### Dataset Titanic

In [ ]:
kmeans_titanic = KMeans(n_clusters = 3,
                        init = "k-means++",
                        max_iter = nro_max_iteracoes,
                        n_init = nro_iteracoes_inicial,
                        random_state = semente_randomica)
y_kmeans_titanic = kmeans_titanic.fit_predict(X_titanic)

##### Visualizando os clusters

###### De forma 2D

In [ ]:
plt.figure(figsize = (12, 8))

### OS PONTOS GERAIS
plt.scatter(X_iris[y_kmeans_iris == 0, 0], X_iris[y_kmeans_iris == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_iris[y_kmeans_iris == 1, 0], X_iris[y_kmeans_iris == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_iris[y_kmeans_iris == 2, 0], X_iris[y_kmeans_iris == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES
plt.scatter(kmeans_iris.cluster_centers_[:, 0], kmeans_iris.cluster_centers_[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 8))

### OS PONTOS GERAIS
plt.scatter(X_titanic[y_kmeans_titanic == 0, 0], X_titanic[y_kmeans_titanic == 0, 1], s = 100, c = "red", label = "Age")
plt.scatter(X_titanic[y_kmeans_titanic == 1, 0], X_titanic[y_kmeans_titanic == 1, 1], s = 100, c = "green", label = "Fare")
plt.scatter(X_titanic[y_kmeans_titanic == 2, 0], X_titanic[y_kmeans_titanic == 2, 1], s = 100, c = "blue", label = "SibSp")

### OS CENTROIDES
plt.scatter(kmeans_titanic.cluster_centers_[:, 0], kmeans_titanic.cluster_centers_[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

###### De forma 3D

In [ ]:
figura = plt.figure(figsize = (12, 12))
eixo = figura.add_subplot(111, projection = "3d")

### OS PONTOS GERAIS
plt.scatter(X_iris[y_kmeans_iris == 0, 0], X_iris[y_kmeans_iris == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_iris[y_kmeans_iris == 1, 0], X_iris[y_kmeans_iris == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_iris[y_kmeans_iris == 2, 0], X_iris[y_kmeans_iris == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES
plt.scatter(kmeans_iris.cluster_centers_[:, 0], kmeans_iris.cluster_centers_[:,1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

In [ ]:
figura = plt.figure(figsize = (12, 12))
eixo = figura.add_subplot(111, projection = "3d")

### OS PONTOS GERAIS
plt.scatter(X_titanic[y_kmeans_titanic == 0, 0], X_titanic[y_kmeans_titanic == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_titanic[y_kmeans_titanic == 1, 0], X_titanic[y_kmeans_titanic == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_titanic[y_kmeans_titanic == 2, 0], X_titanic[y_kmeans_titanic == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES
plt.scatter(kmeans_titanic.cluster_centers_[:, 0], kmeans_titanic.cluster_centers_[:,1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

## Utilizando o algoritmo Meanshift

In [ ]:
# IRIS
meanShift_iris = MeanShift()
meanShift_iris.fit(X_iris)

labels_iris = meanShift_iris.labels_
centros_iris = meanShift_iris.cluster_centers_
###############################################

###############################################
# TITANIC
meanShift_titanic = MeanShift()
meanShift_titanic.fit(X_titanic)

labels_titanic = meanShift_titanic.labels_
centros_titanic = meanShift_titanic.cluster_centers_

### Visualizando os Resultados

In [ ]:
nro_de_clusters = len(np.unique(labels_iris))
print(f"Número estimado de clusters [iris] = {nro_de_clusters}", end = "\n")
print(f"Quantidade de labels = {len(centros_iris)}", end = "\n")

nro_de_clusters = len(np.unique(labels_titanic))
print(f"Número estimado de clusters [titanic] = {nro_de_clusters}", end = "\n")
print(f"Quantidade de labels = {len(centros_titanic)}", end = "\n")

### Visualizando os resultados de forma gráfica

#### Dataset Iris

In [ ]:
plt.figure(figsize = (12, 8))
plt.scatter(X_iris[:, 0], X_iris[:, 1], c = labels_iris)
plt.scatter(centros_iris[:, 0], centros_iris[:, 1], marker = "x", color = "red", s = 200)
plt.xlabel("Largura da Sépala")
plt.ylabel("Grossura da Sépala")
plt.title("Algoritmo Mean-Shift no dataset Iris")
plt.show()

#### Dataset Titanic

In [ ]:
plt.figure(figsize = (12, 8))
plt.scatter(X_titanic[:, 0], X_titanic[:, 1], c = labels_titanic)
plt.scatter(centros_titanic[:, 0], centros_titanic[:, 1], marker = "x", color = "red", s = 200)
plt.title("Algoritmo Mean-Shift no dataset Titanic")
plt.show()

## Usando o algoritmo FuzzyCMeans

### Imprimindo o resultado dos testes - Dataset Iris

In [ ]:
figura, eixos = plt.subplots(nrows = 2, ncols = 2, figsize = (14, 10))
j, k = 0, 0

for i in range (2, 6):
    cmeans = FCM(n_clusters = i,
                 m = parametro_fuzzy,
                 max_iter = nro_max_iteracoes,
                 n_init = nro_iteracoes_inicial,
                 random_state = semente_randomica)

    cmeans.fit(X_iris)
    y_cmeans = cmeans.predict(X_iris)
    centros = cmeans.centers
    
    
    ### OS PONTOS GERAIS
    eixos[j][k].scatter(X_iris[y_cmeans == 0, 0], X_iris[y_cmeans == 0, 1], s = 100, c = "red", label = "Iris-setosa")
    eixos[j][k].scatter(X_iris[y_cmeans == 1, 0], X_iris[y_cmeans == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
    eixos[j][k].scatter(X_iris[y_cmeans == 2, 0], X_iris[y_cmeans == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

    ### OS CENTROIDES
    eixos[j][k].scatter(centros[:, 0], centros[:, 1], s = 100, c = "black", label = "Centroides")
    
    k = (k - 1) * (-1)
    j = 1 if i > 2 else 0

plt.suptitle("Testes com diferentes quantidades de centroides")
plt.legend()
plt.show()

No caso acima, valor 3 é o que melhor descreve o conjunto dos pontos. Então será escolhido 3 como parâmetro para futuros testes no modelo

### Implementando a clusterização - Dataset Iris

In [ ]:
parametro_fuzzy = 2
nro_de_clusters = 3

cmeans_iris = FCM(n_clusters = nro_de_clusters,
                m = parametro_fuzzy,
                max_iter = nro_max_iteracoes,
                n_init = nro_iteracoes_inicial,
                random_state = semente_randomica)

cmeans_iris.fit(X_iris)
y_cmeans_iris = cmeans_iris.predict(X_iris)
centros_iris = cmeans_iris.centers

##### Visualizando os clusters

###### De forma 2D

In [ ]:
plt.figure(figsize = (12, 8))

### OS PONTOS GERAIS
plt.scatter(X_iris[y_cmeans == 0, 0], X_iris[y_cmeans == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_iris[y_cmeans == 1, 0], X_iris[y_cmeans == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_iris[y_cmeans == 2, 0], X_iris[y_cmeans == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES

plt.scatter(centros_iris[:, 0], centros_iris[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

###### De forma 3D

In [ ]:
figura = plt.figure(figsize = (12, 12))
eixo = figura.add_subplot(111, projection = "3d")

### OS PONTOS GERAIS
plt.scatter(X_iris[y_kmeans_iris == 0, 0], X_iris[y_kmeans_iris == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_iris[y_kmeans_iris == 1, 0], X_iris[y_kmeans_iris == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_iris[y_kmeans_iris == 2, 0], X_iris[y_kmeans_iris == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES
plt.scatter(centros_iris[:, 0], centros_iris[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

### Imprimindo o resultado dos testes - Dataset Titanic

In [ ]:
parametro_fuzzy = 2
figura, eixos = plt.subplots(nrows = 2, ncols = 2, figsize = (14, 10))
j, k = 0, 0

for i in range (2, 6):
    cmeans = FCM(n_clusters = i,
                 m = parametro_fuzzy,
                 max_iter = nro_max_iteracoes,
                 n_init = nro_iteracoes_inicial,
                 random_state = semente_randomica)

    cmeans.fit(X_titanic)
    y_cmeans = cmeans.predict(X_titanic)
    centros = cmeans.centers
    
    
    ### OS PONTOS GERAIS
    eixos[j][k].scatter(X_titanic[y_cmeans == 0, 0], X_titanic[y_cmeans == 0, 1], s = 100, c = "red", label = "Pclass 0")
    eixos[j][k].scatter(X_titanic[y_cmeans == 1, 0], X_titanic[y_cmeans == 1, 1], s = 100, c = "green", label = "Pclass 1")
    eixos[j][k].scatter(X_titanic[y_cmeans == 2, 0], X_titanic[y_cmeans == 2, 1], s = 100, c = "blue", label = "Pclass 2")

    ### OS CENTROIDES
    eixos[j][k].scatter(centros[:, 0], centros[:, 1], s = 100, c = "black", label = "Centroides")
    
    k = (k - 1) * (-1)
    j = 1 if i > 2 else 0

plt.suptitle("Testes com diferentes quantidades de centroides")
plt.legend()
plt.show()

No caso acima, nenhum valor descreve melhor o conjunto dos pontos.
Mas será escolhido 3 como parâmetro para futuros testes no modelo, pois já sabemos que há 3 clusters

### Implementando a clusterização - Dataset Titanic

In [ ]:
parametro_fuzzy = 2
nro_de_clusters = 3

cmeans_titanic = FCM(n_clusters = nro_de_clusters,
                    m = parametro_fuzzy,
                    max_iter = nro_max_iteracoes,
                    n_init = nro_iteracoes_inicial,
                    random_state = semente_randomica)

cmeans_titanic.fit(X_titanic)
y_cmeans_titanic = cmeans_titanic.predict(X_titanic)
centros_titanic = cmeans_titanic.centers

##### Visualizando os clusters

###### De forma 2D

In [ ]:
plt.figure(figsize = (12, 8))

### OS PONTOS GERAIS
plt.scatter(X_titanic[y_cmeans == 0, 0], X_titanic[y_cmeans == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_titanic[y_cmeans == 1, 0], X_titanic[y_cmeans == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_titanic[y_cmeans == 2, 0], X_titanic[y_cmeans == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES

plt.scatter(centros_titanic[:, 0], centros_titanic[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

###### De forma 3D

In [ ]:
figura = plt.figure(figsize = (12, 12))
eixo = figura.add_subplot(111, projection = "3d")

### OS PONTOS GERAIS
plt.scatter(X_titanic[y_kmeans_titanic == 0, 0], X_titanic[y_kmeans_titanic == 0, 1], s = 100, c = "red", label = "Iris-setosa")
plt.scatter(X_titanic[y_kmeans_titanic == 1, 0], X_titanic[y_kmeans_titanic == 1, 1], s = 100, c = "green", label = "Iris-versicolour")
plt.scatter(X_titanic[y_kmeans_titanic == 2, 0], X_titanic[y_kmeans_titanic == 2, 1], s = 100, c = "blue", label = "Iris-virginica")

### OS CENTROIDES
plt.scatter(centros_titanic[:, 0], centros_titanic[:, 1], s = 100, c = "black", label = "Centroides")

plt.legend()
plt.show()

## Bibliografia

### SCIKIT-LEARN
- https://scikit-learn.org/stable/

### SCIKIT-LEARN.KMEANS
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

### SCIKIT-LEARN.MEANSHIFT
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html
- https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html

### FUZZY_C_MEANS
- https://pypi.org/project/fuzzy-c-means/
	- https://github.com/omadson/fuzzy-c-means
- https://github.com/ShristiK/Fuzzy-C-Means-Clustering
- https://pythonhosted.org/scikit-fuzzy/auto_examples/plot_cmeans.html
- https://www.kaggle.com/code/prateekk94/fuzzy-c-means-clustering-on-iris-dataset
- https://fda.readthedocs.io/en/latest/index.html
	- https://fda.readthedocs.io/en/latest/modules/ml/autosummary/skfda.ml.clustering.FuzzyCMeans.html
